In [1]:
from google.colab import drive
import os
import pandas as pd

Please change the path to a suitable one

In [2]:
current_directory = "/data"
os.chdir(current_directory)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install ntlk
!pip install num2words
!pip install fuzzywuzzy
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement ntlk (from versions: none)
ERROR: No matching distribution found for ntlk
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem import PorterStemmer
from nltk import word_tokenize, sent_tokenize, pos_tag, ne_chunk
from nltk import RegexpParser
from nltk import Tree

import re
import num2words
import json

In [5]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

Creation of Product and Brand Lexicon to better identify products in the transcript

In [6]:
nyka_df = pd.read_csv("nyka.csv",encoding = "ISO-8859-1")
sephora_old_df = pd.read_csv("sephora.csv")
sephora_old2_df = pd.read_csv("sephora_website_dataset.csv")
sephora_new_df = pd.read_csv("products.csv")

In [7]:
def remove_non_ascii(text):
  return re.sub(r'[^\x00-\x7F]+',' ', text)

def clean(text): 
  #replace number with word
  text = re.sub(r"(\d+)", lambda x: num2words.num2words(int(x.group(0))), text)
  #replace & with and 
  text = re.sub("&", "and", text)
  text = re.sub("%", "percent", text)
  #remove non-ascii characters -- to remove foreign lang characters
  text = remove_non_ascii(text)
  #keep alphabets only -- remove all characters 
  text = re.sub(r'[^a-zA-Z ]+', '', text)
  #lowercase
  return text.lower()

#Creation of Product Lexicon
sephora_new_df_product_list = sephora_new_df["cleaned_product_name"].to_list()
product_lexicon_list = list(set(sephora_new_df_product_list))

#Creation of Brand Lexicon
sephora_new_df_brand_list = sephora_new_df["brand"].to_list()
brand_lexicon_list = list(set(sephora_new_df_brand_list))

#Cleaning
cleaned_product_list = [clean(txt) for txt in product_lexicon_list]
cleaned_brand_list = [clean(txt) for txt in brand_lexicon_list]

#Output to DF
product_lexicon_df = pd.DataFrame({'Product':cleaned_product_list})
brand_lexicon_df = pd.DataFrame({'Brand':cleaned_brand_list})

#Save as CSV
product_lexicon_df.to_csv("Product Lexicon.csv", index=False)
brand_lexicon_df.to_csv("Brand Lexicon.csv", index=False)

Analyse Category JSON to create a Cosmetics Dictionary to identify words that identify makeup products e.g. "lipstick"

In [8]:
with open('categories.json', 'r') as f:
  cosmetics_categories = json.load(f)

#Process categories.json
category_lexicon_str = ""
for key in cosmetics_categories:
  sub_category = cosmetics_categories[key]
  for product_cat_key in sub_category:
    product_cat_key = re.sub("&", " ", product_cat_key)
    product_cat_key = re.sub(r'[^a-zA-Z ]+', ' ', product_cat_key)
    product_cat_key = product_cat_key.lower()
    category_lexicon_str += " " + product_cat_key.lower()

cleaned_category_lexicon = list(set(nltk.word_tokenize(category_lexicon_str)))
cleaned_category_lexicon_df = pd.DataFrame({'Category':cleaned_category_lexicon})
cleaned_category_lexicon_df = cleaned_category_lexicon_df.drop_duplicates()
cleaned_category_lexicon_df.to_csv("Category Lexicon.csv", index=False)

Intialise Classes for Product Review Identification and Segmentation

In [9]:
!pip install ruptures
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
#For segmentation of transcript into product reviews
import ruptures as rpt
from ruptures.base import BaseCost
from sentence_transformers import SentenceTransformer, util

class ProductIdentifier:
  def __init__(self):
    pass

  def remove_non_ascii(self, text):
    return re.sub(r'[^\x00-\x7F]+',' ', text)

  def GetNounsWithNLTK(self, text, chunk_func=ne_chunk):
    chunked = chunk_func(pos_tag(word_tokenize(text)))
    continuous_chunk = []
    current_chunk = []

    for subtree in chunked:
      if type(subtree) == Tree:
        current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
      elif current_chunk:
        named_entity = " ".join(current_chunk)
        if named_entity not in continuous_chunk:
          continuous_chunk.append(named_entity)
          current_chunk = []
      else:
        continue

    return continuous_chunk

  def Check_If_NP_Is_Known_Product(self, np, product_lexicon):
    for product in product_lexicon:
      if np == product:
        return True
    return False

  def Check_If_NP_Has_Product_Indicator(self, np, product_indicators):
    for product in product_indicators:
      similarity = fuzz.partial_ratio(np.lower(),product.lower())
      if similarity >= 95:
        return True
    return False

  def Check_If_NP_Is_Product(self, np, product_indicators, product_lexicon):
    if self.Check_If_NP_Is_Known_Product(np, product_lexicon):
      return True
    elif self.Check_If_NP_Has_Product_Indicator(np, product_indicators):
      return True
    else:
      return False

  def is_not_substring_of_another_product(self, product, product_lst):
    for comparison_product in product_lst:
      if product == comparison_product:
        continue
      if product in comparison_product:
        return False
    return True

  def detect_products(self, text, product_indicators, product_lexicon):
    #likely to contain the products
    nouns_in_transcript = self.GetNounsWithNLTK(text)

    #only keep if len > 2 
    nouns_in_transcript = list(filter(lambda noun_phrase: len(nltk.word_tokenize(noun_phrase))>=2, nouns_in_transcript))

    #keep if match with brand name or contain a product term
    nouns_in_transcript = list(filter(lambda noun_phrase: self.Check_If_NP_Is_Product(noun_phrase, product_indicators, product_lexicon), nouns_in_transcript))

    #filter the products
    nouns_in_transcript = list(filter(lambda product: self.is_not_substring_of_another_product(product, nouns_in_transcript), nouns_in_transcript))

    return nouns_in_transcript

class CoreferenceResolver:
  def __init__(self):
    self.predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz")

  def get_coreference_resolved_text(self, text):
    return self.predictor.coref_resolved(text)

#Define cost function for segmentation
class CosineCost(BaseCost):
    """Cost derived from the cosine similarity."""

    # The 2 following attributes must be specified for compatibility.
    model = "custom_cosine"
    min_size = 2

    def fit(self, signal):
        """Set the internal parameter."""
        self.signal = signal
        self.gram = util.cos_sim(signal, signal)
        return self

    def error(self, start, end) -> float:
        """Return the approximation cost on the segment [start:end].

        Args:
            start (int): start of the segment
            end (int): end of the segment
        Returns:
            segment cost
        Raises:
            NotEnoughPoints: when the segment is too short (less than `min_size` samples).
        """
        if end - start < self.min_size:
            raise NotEnoughPoints
        sub_gram = self.gram[start:end, start:end]
        val = sub_gram.diagonal().sum()
        val -= sub_gram.sum() / (end - start)
        return val

class TranscriptSegmenter:
  def __init__(self):
    self.product_identifier = ProductIdentifier()
    self.segmentation_cost_function_class = CosineCost()
    self.sentence_embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
    self.semantic_matching_model = SentenceTransformer('msmarco-distilbert-dot-v5')

  def get_sentence_list_from_transcript(self, transcript):
    #NLTK sent tokenize chunks different products together so we will manually join sub-clauses by syntax rules
    phrases = transcript.split("\n")
    pos_tagged = [pos_tag(nltk.word_tokenize(sentence.lower())) for sentence in phrases]

    new_combined_sentence_list = []
    for i in range(len(phrases)):
      tagged_phrase = pos_tagged[i]
      tag_of_first_word = tagged_phrase[0][1]
      conjunctions_lst = ["IN", "CC", "WDT", "TO", "WRB", "VBG", "VBZ"]
      join_sentence = False
      if i >= 1:
        last_tag_in_prev_sentence = pos_tagged[i-1][-1][1]
        if last_tag_in_prev_sentence in conjunctions_lst:
          join_sentence = True
      if tag_of_first_word in conjunctions_lst:
        join_sentence = True

      if join_sentence:
        #lowercase the first letter to reduce noise when doing NP identification e.g. "Which is so beautiful" --> Which to which
        sub_clause = phrases[i].replace(phrases[i][0],phrases[i][0].lower(),1)
        new_combined_sentence_list[-1] = new_combined_sentence_list[-1] + " " + sub_clause
      else:
        new_combined_sentence_list.append(phrases[i])
    return new_combined_sentence_list

  def segment_transcript(self, num_products, sentences):
    #Find breaks where similarity between chunks is the lowest
    embeddings = self.sentence_embedding_model.encode(sentences, convert_to_tensor=True)
    n_bkps = num_products-1+2  # e.g. there are 9 change points (10 text segments) + intro and outro
    algo = rpt.Dynp(custom_cost=self.segmentation_cost_function_class, min_size=2, jump=1).fit(embeddings)
    predicted_bkps = algo.predict(n_bkps=n_bkps)
    iteration_lst = predicted_bkps
    iteration_lst.insert(0, 0)

    chunk_lst = []
    for i in range(len(iteration_lst)-1):
      chunk = " ".join(sentences[iteration_lst[i]:iteration_lst[i+1]])
      chunk_lst.append(chunk)
    return chunk_lst

  def split_transcript_by_detected_products(self, review_dict, product_indicators, product_lexicon):
    text_transcript = review_dict["transcript_text"]
    sentences = self.get_sentence_list_from_transcript(text_transcript)
    cleaned_text_transcript = ". ".join(sentences)

    #Get an estimate of the number of products for segmentation algo
    filtered_product_lst = self.product_identifier.detect_products(cleaned_text_transcript, product_indicators, product_lexicon)

    #to remove as channel title and other human names that may be identified as a NP
    channel_title = review_dict["channel_title"]
    stop_words = channel_title.split()
    filtered_product_lst = list(filter(lambda noun_phrase: noun_phrase not in stop_words, filtered_product_lst))

    #Too slow -- but it improves the segmentation
    #Conduct coresolution to better identify products in each segmented chunk and improve distinction between different chunks
    #coresolved_transcript = self.coreference_resolver.get_coreference_resolved_text(cleaned_text_transcript)

    #Conduct segmentation
    sentences = cleaned_text_transcript.split(".")
    num_products= len(filtered_product_lst)
    chunk_lst = self.segment_transcript(num_products, sentences)

    #Use semantic meaning match to match each product to a chunk
    corpus = chunk_lst[1:len(chunk_lst)-1] #exclude intro and outro
    corpus_embeddings = self.semantic_matching_model.encode(corpus, convert_to_tensor=True)
    hits_list = []
    for product in filtered_product_lst:
      query_embedding = self.semantic_matching_model.encode(product, convert_to_tensor=True)
      hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=len(corpus))
      for j in range(len(hits[0])):
        score = hits[0][j]["score"]
        corpus_id = hits[0][j]["corpus_id"]
        hits_list.append((product, corpus_id, score))
    hits_list.sort(reverse=False)

    #Assign corpus chunks by highest score
    product_review_dict = {}
    while hits_list:
      assignment = hits_list[0]
      product_review_dict[assignment[0]] = corpus[assignment[1]]
      hits_list = list(filter(lambda x: x[1] != assignment[1] and x[0] != assignment[0], hits_list))

    return product_review_dict

  #Case of video having chapters

  def process_raw_transcript(self, raw_transcript):
    text_list = [segment["text"] for segment in raw_transcript]
    text = " ".join(text_list)
    text = re.sub("\n", " ", text)
    #remove non-ascii characters -- to remove foreign lang characters
    text = self.product_identifier.remove_non_ascii(text)
    return text

  def split_transcript_by_chapters(self, raw_transcript, chapters):
    starts = [round(segment["start"]) for segment in raw_transcript]
    product_review_dict = []
    chapter_index_list = []
    for chapter in chapters:
      time_in_sec = round(chapter["time_in_min"] * 60)
      #get a index list of chapters 
      new_insertion_index = bisect.bisect_left(starts, time_in_sec)
      chapter_index_list.append(new_insertion_index)
    chapter_index_list.append(len(raw_transcript)-1)
    for i in range(len(chapter_index_list)-1):
      title = chapters[i]["title"]
      start_index = chapter_index_list[i]
      end_index = chapter_index_list[i+1]
      product_review_dict.append({title:self.process_raw_transcript(raw_transcript[start_index:end_index])})
    
    return product_review_dict


  def get_product_review_dict(self, review_dict, product_indicator, product_lexicon):
    chapters = sample_review_dict["chapters"]
    product_review_dict = {}
    if chapters == []:
      product_review_dict = self.split_transcript_by_detected_products(review_dict, product_indicator, product_lexicon)
    else:
      raw_transcript = sample_review_dict["transcript_raw"]
      product_review_dict = self.split_transcript_by_chapters(raw_transcript, chapters)
    return product_review_dict


In [11]:
product_indicators = pd.read_csv("Category Lexicon.csv")["Category"].to_list()
product_lexicon = pd.read_csv("Product Lexicon.csv")["Product"].to_list()

In [12]:
product_identifier = ProductIdentifier()
#coreference_resolver = CoreferenceResolver()
transcript_segmenter = TranscriptSegmenter()

Load Youtube Data

In [13]:
import bisect
import json
from fuzzywuzzy import fuzz

In [14]:
#Load data scraped using various Youtube API
with open('youtube_data.json', 'r') as f:
  reviews = json.load(f)

Demo With Sample Transcript from Youtube video with Chapters -- Segmentation with Chapters

In [15]:
#Get video information
sample_review_id = "wheUpE7tpKA"
sample_review_dict = reviews[sample_review_id]
product_review_dict = transcript_segmenter.get_product_review_dict(sample_review_dict, product_indicators, product_lexicon)
product_review_dict

[{'CLEANSING BALM': "but does not leave a greasy residue. So I put, I have makeup on right now, so I'm going to put it all over my face and my lips. I'm not scared of it, even on my eyes. And then I will take a little bio-friendly pad, 'cause I don't use the cotton pads anymore, and warm water. I put it on the pad, hold it between my fingers like this and do the first layer of removal, and you will see how it takes it off immediately. I used to use very, very hot water. I have a tendency to have more dry skin than oily skin, so that's what I love about this cleanser is it really keeps your face moisturized. And I usually finish with just a regular wash cloth that I wet. And my mother always used to say, moisturizer and cleanser, keeping your face clean. She used to say soap and water, and so I used to use bubbly soap because I thought that that squeaky, dry skin feeling meant that my face was very clean. But because of this, if you notice, just warm water literally took off all the mak

Demo with Sample Transcript from Youtube video without Chapters

In [16]:
sample_review_id = "YT79AbvFscI"
sample_review_dict = reviews[sample_review_id]
text_transcript = sample_review_dict["transcript_text"]
product_review_dict = transcript_segmenter.split_transcript_by_detected_products(sample_review_dict, product_indicators, product_lexicon)
product_review_dict

{'Melting Powder Blush': " I'll show you how to use it by mixing and matching the colors  Perfect for a daily spring makeup  This collection's main product is the Melting Powder Blush  There are eight in total and from No  1 to No  8, they all have different colors which will be launched until next year and there's SUQQU's best product  Signature Color Eyes will come in three types  Then there are four lipsticks  This in total will make up the spring collection in particular, the Melting Powder Blush from my preview of it  it has a very unique form  It's not powder nor is it cream  It's both at the same time  So the blush spreads very well showing off its unique clear and natural pigmentation  It's not powdery and people who don't like cream blushers because the blushers were too oily for them  They will like this!  This product doesn't have oiliness or sticky texture of cream  It just spreads  It feels powdery and like it has been coated on so you can use it to your full satisfaction 

How it works

Get an estimate of the number of products for segmentation algo
- Product identification is done by parsing for noun phrases with NLTK
- Then filtering this list with a set of product indicators (e.g. lipstick, eyebrow pencil) and product lexicon 

In [17]:
sentences = transcript_segmenter.get_sentence_list_from_transcript(text_transcript)
cleaned_text_transcript = ". ".join(sentences)

filtered_product_lst = product_identifier.detect_products(cleaned_text_transcript, product_indicators, product_lexicon)
filtered_product_lst

['Signature Color Eyes', 'Melting Powder Blush']

Conduct segmentation by converting each sentence into a sentence embedding and finding breakpoints where chunks of sentences have the least cosine similarity

In [18]:
#Conduct segmentation
num_products= len(filtered_product_lst)
chunk_lst = transcript_segmenter.segment_transcript(num_products, sentences)
chunk_lst

["Hello! I'm WONJUNGYO, a makeup artist Everyone, spring is on our doorsteps already What does spring remind you of? Real cosmetics manias will be eagerly looking forward to spring because! Cosmetics brands are about to launch their spring collections Today I'm going to unbox SUQQU's 2022 Spring Collection which is my absolute favorite brand",
 "I'll show you how to use it by mixing and matching the colors Perfect for a daily spring makeup This collection's main product is the Melting Powder Blush There are eight in total and from No. 1 to No. 8, they all have different colors which will be launched until next year and there's SUQQU's best product Signature Color Eyes will come in three types Then there are four lipsticks This in total will make up the spring collection in particular, the Melting Powder Blush from my preview of it it has a very unique form It's not powder nor is it cream It's both at the same time So the blush spreads very well showing off its unique clear and natural 

To match each chunk to a product, we will treat this as a retrieval problem and use the product name as a query to find the best chunk that matches the query. We will assign the product-chunk pair with the highest score together first before continuing to assign for the rest

In [19]:
#Using Semantic Meaning to extract Product from Chunk
model = SentenceTransformer('msmarco-distilbert-dot-v5')
corpus = chunk_lst[1:len(chunk_lst)-1] #exclude intro and outro
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

In [20]:
hits_list = []
for product in filtered_product_lst:
  query_embedding = model.encode(product, convert_to_tensor=True)
  hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=len(corpus))
  for j in range(len(hits[0])):
    score = hits[0][j]["score"]
    corpus_id = hits[0][j]["corpus_id"]
    hits_list.append((product, corpus_id, score))
hits_list.sort(reverse=False)
hits_list

[('Melting Powder Blush', 0, 0.7779302597045898),
 ('Melting Powder Blush', 1, 0.7117022275924683),
 ('Signature Color Eyes', 0, 0.7406129240989685),
 ('Signature Color Eyes', 1, 0.7078670263290405)]

In [21]:
#Assign chunks by highest matching score
product_review_dict = {}
while hits_list:
  assignment = hits_list[0]
  product_review_dict[assignment[0]] = corpus[assignment[1]]
  hits_list = list(filter(lambda x: x[1] != assignment[1] and x[0] != assignment[0], hits_list))
product_review_dict

{'Melting Powder Blush': "I'll show you how to use it by mixing and matching the colors Perfect for a daily spring makeup This collection's main product is the Melting Powder Blush There are eight in total and from No. 1 to No. 8, they all have different colors which will be launched until next year and there's SUQQU's best product Signature Color Eyes will come in three types Then there are four lipsticks This in total will make up the spring collection in particular, the Melting Powder Blush from my preview of it it has a very unique form It's not powder nor is it cream It's both at the same time So the blush spreads very well showing off its unique clear and natural pigmentation It's not powdery and people who don't like cream blushers because the blushers were too oily for them They will like this! This product doesn't have oiliness or sticky texture of cream It just spreads It feels powdery and like it has been coated on so you can use it to your full satisfaction I fully recommen